# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd #импортируем билбиотеку Pandas

In [2]:
df = pd.read_csv('/data/data.csv') #Читаем dataset и сохраняем его в переменной df

In [3]:
df.head(30) #выводим первые 10 строк таблицы c данными

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
df.info() #получаем общую информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

**Выгрузив таблицу мы обнаружили 12 столбцов и 3 типа данных: `float64`, `int64`, `object`.**

* `children` - количество детей в семье заёмщика;
* `days_employed` - общий трудовой стаж в днях заёмщика;
* `dob_years` - возраст заёмщика в годах;
* `education` -  уровень образования заёмщика;
* `education_id` - идентификатор уровня образования заёмщика;
* `family_status` - семейное положение заёмщика;
* `family_status_id` - идентификатор семейного положения заёмщика;
* `gender` - пол заёмщика;
* `income_type` -  тип занятости заёмщика;
* `debt` - имел ли заёмщик задолженность по возврату кредитов;
* `total_income` - ежемесячный доход заёмщика;
* `purpose` - цель получения кредита;

   **В столбце `days_employed` нашли отрицательные значения.
   В столбце `education` строчные буквы встречаются с прописными.
   Так же обнаружены пропуски в столбцах: `days_employed`, `total_income`.**
   
   **Для дальнейшего продвижения нужно устранить проблемы в данных.**




## Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
df.isna().sum() #находим пропущенные значения(где и сколько)

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Обнаружены пропуски в столбцах: `days_employed`, `total_income`.
Но сначала мы изменим отрицательные значения на положительные, далее займемся заменой пропусков.**

**Пишем функцию для устранения отрицательных значений.**

In [6]:
def removing_the_minus(i): #В столбце `days_employed` изменим отрицательные значение на положительные. Создадим функцию removing_the minus.
    if i < 1:
        i = i * (-1)
        return i
    return i

**Применим функцию `removing_the_minus` к столбцу `days_employed`**

In [7]:
df['days_employed'] = df['days_employed'].apply(removing_the_minus)

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Хороший способ изменить отрицательные значения на положительные.
    
Мы также можем использовать готовый метод `abs`, оставил пример ниже.
</div>

In [8]:
df['days_employed'].abs()

0          8437.673028
1          4024.803754
2          5623.422610
3          4124.747207
4        340266.072047
             ...      
21520      4529.316663
21521    343937.404131
21522      2113.346888
21523      3112.481705
21524      1984.507589
Name: days_employed, Length: 21525, dtype: float64

**Заменяем пропуски медианой (среднее).**

In [9]:
df.loc[df['days_employed'].isna(), 'days_employed'] = df['days_employed'].median() #Медиана для столбца `days_employed` (делаем через `loc` по совету в методичке)
df.loc[df['total_income'].isna(), 'total_income'] = df['total_income'].median() #Медиана для столбца `total_income` (делаем через `loc` по совету в методичке)

In [10]:
df.isna().sum() # проверяем таблицу на отсутствие пропусков(пропусков нет)

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Пропуски устранены.**

In [11]:
df.head(30) #проверяем таблицу на исправления отрицательных значений(значения исправлены)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Есть аномально большие значения в столбце `days_employed`, с такими значенями работать не удобно. Есть предположение, что это дни. Переводим их в года.**

In [12]:
def transform_in_years(i): #Пишем функцию для перевода дней в столбце `days_employed` в года
    i = i/365
    return i


In [13]:
df['days_employed'] = df['days_employed'].apply(transform_in_years) #применяем функцию 

In [14]:
df.head(30) # Проверяем результат

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,932.235814,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Некоторые значения так и остались аномально большими. Появилось предположение, что эти данные могли быть взяты из другой системы. Пишем еще одну функцию.**

In [15]:
def into_years(i):
    if i > 100:
        i = i / 24
    return i

In [16]:
df['days_employed'] = df['days_employed'].apply(into_years) #применяем функцию

In [17]:
df.head(30) #проверяем результат

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23.116912,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,11.026860,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,15.406637,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,11.300677,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,38.843159,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,2.537495,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,7.888225,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,0.418574,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,18.985932,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,5.996593,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

* в столбце `days_employed` все отрицательные значения заменены на положительные, заменены все пропуски медианой данного столбца. А так же исправлен стаж работы (ранее исчислялся для некоторых заёмщиков в днях или часах) на года.
* в столбце `total_income` мы так же заменили все пропуски.**

* 2174 заёмщика не указали свой стаж работы и уровень дохода. Вероятнее всего, они умышленно не заполняли эти данные (работа с серой зарплатой)**

### Замена типа данных

In [18]:
df.info() #проверяем какие типы данных присутствуют в таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Все столбцы имеют корректный тип данных. Но столбцы `days_employed` (отработанные дни) и `total_income`(общий доход) указаны с дробной частью (`float`).Переведем их в `int`.**

In [19]:
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype(int) #применяем метод astype() для изменения типа float на int.

In [20]:
df.info() #проверяем

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int32 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int32 
 11  purpose           21525 non-null  object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


**Тип данных изменился**

In [21]:
df.head(30) # столбцы `days_employed` и `total_income` приведены к целочисленному типу int.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,15,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,18,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

* в столбцах `days_employed`(общий трудовой стаж в днях заёмщика) и `total_income`(ежемесячный доход заёмщика) мы наблюдаем тип `float`.**

* для удобства чтения и дальнейших манипуляций с этими данными нам надо привести тип данных этих столбцов к целочисленным(`int`).**

* так как никто не говорит, что стаж равен 45.694194, а доход равен 267628.550329.**

* был выбран метод astype(), так как этот метод подходит для перевода в целые числа.**

### Обработка дубликатов

**Выводим на экран подсчет уникальных значений столбцов: `education`, `family_status`, `gender`, `income_type`, `purpose`, `children` методом value_counts()**

In [22]:
display(df['education'].value_counts())  

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [23]:
display(df['family_status'].value_counts()) 

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [24]:
display(df['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [25]:
display(df['income_type'].value_counts())

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [26]:
display(df['purpose'].value_counts())

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

In [27]:
display(df['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

**Приводим значения столбцов `education`, `family_status` к строчному типу методом `str.lower()`**

In [28]:
df['education'] = df['education'].str.lower() 

In [29]:
df['family_status'] = df['family_status'].str.lower()

**В столбце `children` исправляем аномальные значения методом `replace`**

In [30]:
df['children'] = df['children'].replace(-1, 1)

In [31]:
df['children'] = df['children'].replace(20, 2)

In [32]:
display(df['education'].value_counts()) #проверяем

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [33]:
display(df['family_status'].value_counts()) # проверяем

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

**Значения приведены к строчному типу**

In [34]:
display(df['children'].value_counts())

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Аномальные значения исправлены**

In [35]:
df.duplicated().sum() #считаем дубликаты методом sum()

71

Далее удаляем дубликаты из датафрейма методом `drop_duplicates()`

In [36]:
df = df.drop_duplicates().reset_index(drop=True)

In [37]:
df.duplicated().sum() #проверяем

0

**Вывод**

* Дубликаты удалены
* Аномальные значения справлены на корректные
* В столбцах `education` и `family_status` значения приведены к строчному типу

### Лемматизация

**Проанализируем нашу таблицу. Следующим шагом будет лемматизация столбца `purpose`. В остальных она не требуется.**

**Импортируем библиотеку `pymystem3` для дальнейшей работы с лемматизацей.**

In [38]:
from pymystem3 import Mystem # импортируем библиотеку с функцией лемматизации на русском языке
m = Mystem() # присвоим переменную m

**Найдем уникальные значения столбца `purpose`**

In [39]:
text = df['purpose'].unique()
text

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Далее склеим в одну строчку, методом `join()`** 

In [40]:
text_str = ','.join(text)
text_str

'покупка жилья,приобретение автомобиля,дополнительное образование,сыграть свадьбу,операции с жильем,образование,на проведение свадьбы,покупка жилья для семьи,покупка недвижимости,покупка коммерческой недвижимости,покупка жилой недвижимости,строительство собственной недвижимости,недвижимость,строительство недвижимости,на покупку подержанного автомобиля,на покупку своего автомобиля,операции с коммерческой недвижимостью,строительство жилой недвижимости,жилье,операции со своей недвижимостью,автомобили,заняться образованием,сделка с подержанным автомобилем,получение образования,автомобиль,свадьба,получение дополнительного образования,покупка своего жилья,операции с недвижимостью,получение высшего образования,свой автомобиль,сделка с автомобилем,профильное образование,высшее образование,покупка жилья для сдачи,на покупку автомобиля,ремонт жилью,заняться высшим образованием'

**Производим лемматизацию строки `text_str`**

In [41]:
lemmas = m.lemmatize(text_str)
lemmas

['покупка',
 ' ',
 'жилье',
 ',',
 'приобретение',
 ' ',
 'автомобиль',
 ',',
 'дополнительный',
 ' ',
 'образование',
 ',',
 'сыграть',
 ' ',
 'свадьба',
 ',',
 'операция',
 ' ',
 'с',
 ' ',
 'жилье',
 ',',
 'образование',
 ',',
 'на',
 ' ',
 'проведение',
 ' ',
 'свадьба',
 ',',
 'покупка',
 ' ',
 'жилье',
 ' ',
 'для',
 ' ',
 'семья',
 ',',
 'покупка',
 ' ',
 'недвижимость',
 ',',
 'покупка',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ',',
 'покупка',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ',',
 'строительство',
 ' ',
 'собственный',
 ' ',
 'недвижимость',
 ',',
 'недвижимость',
 ',',
 'строительство',
 ' ',
 'недвижимость',
 ',',
 'на',
 ' ',
 'покупка',
 ' ',
 'подержать',
 ' ',
 'автомобиль',
 ',',
 'на',
 ' ',
 'покупка',
 ' ',
 'свой',
 ' ',
 'автомобиль',
 ',',
 'операция',
 ' ',
 'с',
 ' ',
 'коммерческий',
 ' ',
 'недвижимость',
 ',',
 'строительство',
 ' ',
 'жилой',
 ' ',
 'недвижимость',
 ',',
 'жилье',
 ',',
 'операция',
 ' ',
 'со',
 ' ',
 'свой',
 ' ',
 'недвижимо

**Далее с помощью `Counter` посчитаем переодичность появления слов. Для этого вызовем контейнер `Counter` из модуля ` collections`.**

In [42]:
from collections import Counter # Вызов Counter

In [43]:
Counter(lemmas) # Подсчет колличества каждого слова в lemmas

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         ',': 37,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         '\n': 1})

**После выделим вручную из этого списка часто повторяющиеся существительные - это будут основные категории**

**А именно: жилье, недвижимость, автомобиль, образование, операция, свадьба.**

**Напишем функцию для `apply`, которая лемматизирует последовательно каждую ячейку столбца `purpose` и проверит, какая из категорий обнаруживается в лематезированном варианте.**

In [44]:
def lemmatize(purpose):
    if 'жил' in purpose:
        return 'жилье'
    if 'недвиж' in purpose:
        return 'неджижимость'
    if 'авто' in purpose:
        return 'автомобиль'
    if 'образ' in purpose:
        return 'образование'
    if 'опер' in purpose:
        return 'операция'
    if 'свад' in purpose:
        return 'свдьба'

**К созданному новому столбцу с категориями `purpose_category` применим функцию `lemmatize`**

In [45]:
df['purpose_category'] = df['purpose'].apply(lemmatize) 

In [46]:
df.head(50) # проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свдьба
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свдьба
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


**Вывод**

* Была произведена лемматизация для целей получения кредита
* Выделены основные на мой взгляд категории для получения кредита
* Создан новый столбец `purpose_category` с основными целями получения кредита

### Категоризация данных

**Категоризируем семейное положение и доход**

In [47]:
df['family_status'].value_counts() # делаем подсчет уникальных значений столбца family_status

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

**Создадим новую таблицу:**

In [48]:
category_family_status = df[['family_status_id', 'family_status']] #новая таблица с идентификатором и семейным положением заёмщиков
category_family_status = category_family_status.drop_duplicates().reset_index(drop=True) # удаляем дубликаты
category_family_status

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


**Категория для дохода**

**Выведем минимальный, средний и максимальный месячный доход:**

In [49]:
display(df['total_income' ].min())
display(df['total_income'].median())
display(df['total_income'].max())

20667

145017.0

2265604

**Напишем функцию для классифакации доходов:**

In [50]:
def profit(total_income):
    if total_income <= 50000:
        return 'низкий доход'
    if total_income <= 150000:
        return 'средний доход'
    return 'высокий доход'
df['income_category'] = df['total_income'].apply(profit)

In [51]:
display(df.head(10)) #проверяем датафрейм

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category
0,1,23,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье,высокий доход
1,1,11,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,средний доход
2,0,15,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье,средний доход
3,3,11,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,высокий доход
4,0,38,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свдьба,высокий доход
5,0,2,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье,высокий доход
6,0,7,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье,высокий доход
7,0,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,средний доход
8,2,18,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свдьба,средний доход
9,0,5,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье,средний доход


**Вывод**

**Были прокатегоризированы:**
* Семейный статус и его его id
* Доходы(Создан новый столбец `income_category` с классификацией уровней доходов)

## Шаг 3. Ответим на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [52]:
df.groupby('children')['debt'].sum()

children
0    1063
1     445
2     202
3      27
4       4
5       0
Name: debt, dtype: int64

**Вывод**

* **Судя по групирровке данных по количеству детей и задолженности, чем больше детей в семье, тем меньше задолженность по кредиту.**

In [53]:
relationship_of_children = df.groupby('children').agg({'debt':['mean','count']})
# мультииндекс переменуем так:
relationship_of_children = relationship_of_children.rename(columns={'debt':'Зависимость'}, level = 0)
relationship_of_children = relationship_of_children.rename(columns={'mean':'% Невозврата', 'count':'Кол-во заёмщиков'}, level = 1)
# индекс переменуем вот так:
relationship_of_children.index = relationship_of_children.index.rename('Кол-во детей')
#сменим формат числа через style.format
relationship_of_children.style.format({('Зависимость', '% Невозврата'):'{:.2%}', ('Зависимость', 'Кол-во заёмщиков'):'{:.2f}',})

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [54]:
df.groupby('family_status')['debt'].sum()

family_status
в разводе                 85
вдовец / вдова            63
гражданский брак         388
женат / замужем          931
не женат / не замужем    274
Name: debt, dtype: int64

In [55]:
dependence_on_income = df.groupby(['family_status_id', 'family_status']).agg({'debt':['mean','count']})
dependence_on_income = dependence_on_income.rename(columns={'debt':'Зависимость'}, level = 0)
dependence_on_income = dependence_on_income.rename(columns={'mean':'% Невозврата', 'count':'Кол-во заёмщиков'}, level = 1)
dependence_on_income.index = dependence_on_income.index.rename(['id с/п', 'семейное положение'])
dependence_on_income.style.format({('Зависимость', '% Невозврата'):'{:.2%}', ('Зависимость', 'Кол-во заёмщиков'):'{:.2f}',})

**Вывод**

* **Исходя из данных вдоцы(вдовы) возвращают кредит чаще всех прочих из этой категории. Те, кто состоит в браке реже всех возвращают кредит в срок.**

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [56]:
df.groupby('income_category')['debt'].sum()

income_category
высокий доход    726
низкий доход      23
средний доход    992
Name: debt, dtype: int64

In [57]:
report = df.groupby('income_category').agg({'debt':['mean','count']})
report = report.rename(columns={'debt':'Зависимость'}, level = 0)
report = report.rename(columns={'mean':'% Невозврата', 'count':'Кол-во заёмщиков'}, level = 1)
report.index = report.index.rename('уровень дохода')
report.style.format({('Зависимость', '% Невозврата'):'{:.2%}', ('Зависимость', 'Кол-во заёмщиков'):'{:.2f}',})

**Вывод**

* **Исходя из данных заёмщики с низким доходом возвращают кредит чаще, чем заёмщики со средним и высоким доходом.**

- Как разные цели кредита влияют на его возврат в срок?

In [58]:
df.groupby('purpose_category')['debt'].sum()

purpose_category
автомобиль      403
жилье           397
неджижимость    385
образование     370
свдьба          186
Name: debt, dtype: int64

In [59]:
report = df.groupby('purpose_category').agg({'debt':['mean','count']})
report = report.rename(columns={'debt':'Зависимость'}, level = 0)
report = report.rename(columns={'mean':'% Невозврата', 'count':'Кол-во заёмщиков'}, level = 1)
report.index = report.index.rename('цели кредита')
report.style.format({('Зависимость', '% Невозврата'):'{:.2%}', ('Зависимость', 'Кол-во заёмщиков'):'{:.2f}',})

**Вывод**

* **Наиболее надёжными категориями для банка являются: жилье и недвижимость. Самая большая просрочка в категориях: автомобиль, образование.**

## Шаг 4. Общий вывод

**Изходя из проанализированных данных мы обнаружили, что:**
* Данные имеют 2174 пропуска
* Есть некорректные значения в доходах и стаже работы заёмщиков
* Есть повторящиющиеся значения(дубликаты)
* Мы привели к словарным значениям некорректно записанные категории
* Категоризированы данные по классификациям дохода заёмщиков(низкий доход, средний доход, высокий доход)

**На основании проанализированных данных мы делаем вывод:**
* есть прямая зависимость между наличием детей и возвратом кредита в срок, а именно: многодетные семьи имеют меньше задолженность по кредиту
* вдоцы(вдовы) возвращают кредит чаще всех прочих из этой категории. Те, кто состоит в браке реже всех возвращают кредит в срок
* заёмщики с низким доходом возвращают кредит чаще, чем заёмщики со средним и высоким доходом
* Наиболее надёжными категориями для банка являются: жилье и недвижимость. Самая большая просрочка в категориях: автомобиль, образование.

